In [1]:
import os
os.environ["NVIDIA_API_KEY"] = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"

In [2]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
llm_nvidia = ChatNVIDIA(model="mistralai/mixtral-8x7b-instruct-v0.1")

In [3]:
from langchain_community.chat_models import ChatOllama

llm_ollama = ChatOllama(model="llama3")

In [4]:
llm = llm_nvidia

# Chains

The chain usually combines a LLM together with a prompt and put a bunch of building blocks together to carry out a sequence of operations on your text/data.

In [5]:
import pandas as pd

product_review = {
    "product": ["Queen Size Sheet Set",
                "Waterproof Phone Pouch",
                "Luxury Air Mattress",
                "Pillows Insert",
                "Milk Frother Handheld",
                "L'Or Espresso Café",
                "Hervidor de Agua Eléctrico",
               ],
    "review":[
        "I ordered a king size set. My only criticism would be that I wish seller would offer the king size set with 4 pillowcases. I separately ordered a two pack of pillowcases so I could have a total of four. When I saw the two packages, it looked like the color did not exactly match. Customer service was excellent about sending me two more pillowcases so I would have four that matched. Excellent! For the cost of these sheets, I am satisfied with the characteristics and coolness of the sheets.",
        "I loved the waterproof sac, although the opening was made of a hard plastic. I don’t know if that would break easily. But I couldn’t turn my phone on, once it was in the pouch.",
        "This mattress had a small hole in the top of it (took forever to find where it was), and the patches that they provide did not work, maybe because it's the top of the mattress where it's kind of like fabric and a patch won't stick. Maybe I got unlucky with a defective mattress, but where's quality assurance for this company? That flat out should not happen. Emphasis on flat. Cause that's what the mattress was. Seriously horrible experience, ruined my friend's stay with me. Then they make you ship it back instead of just providing a refund, which is also super annoying to pack up an air mattress and take it to the UPS store. This company is the worst, and this mattress is the worst.",
        "This is the best throw pillow fillers on Amazon. I’ve tried several others, and they’re all cheap and flat no matter how much fluffing you do. Once you toss these in the dryer after you remove them from the vacuum sealed shipping material, they fluff up great",
        "I loved this product. But they only seem to last a few months. The company was great replacing the first one (the frother falls out of the handle and can't be fixed). The after 4 months my second one did the same. I only use the frother for coffee once a day. It's not overuse or abuse. I'm very disappointed and will look for another. As I understand they will only replace once. Anyway, if you have one good luck.",
        "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur... Vieux lot ou contrefaçon !?",
        "Está lu bonita calienta muy rápido, es muy funcional, solo falta ver cuánto dura, solo llevo 3 días en funcionamiento."        
    ]
}


In [6]:
df = pd.DataFrame(product_review)

In [7]:
df.head()

,product,review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld,I loved this product. But they only seem to la...


## LLMChain

In [8]:
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [9]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}? Give me only one name"
)

In [10]:
chain = LLMChain(llm=llm, prompt=prompt)

/home/mojo/miniconda3/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [11]:
product = "Queen Size Sheet Set"
chain.run(product)

/home/mojo/miniconda3/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


' "Monarch Regalia Linens" could be a fitting name for a company that makes queen-size sheet sets. The name "Monarch" suggests royalty and grandeur, while "Regalia" refers to ceremonial attire or accessories, which could be interpreted as the high-quality, luxurious nature of the sheet sets. Additionally, "Linens" clearly communicates the company\'s product offerings. The name as a whole suggests that the company provides luxurious, high-quality bedding for those who desire a regal sleeping experience.'

## Sequential Chains

![Sequential Chain](data/images/sequential-chain.png)

### Simple Sequential Chain

![Simple Sequential Chain](data/images/simple-sequential-chain.png)

In [12]:
from langchain.chains import SimpleSequentialChain

In [13]:
first_prompt = ChatPromptTemplate.from_template(
    "Give me only the best name to describe the company that makes {product}."
)

chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [14]:
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following company {company}"
)

chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [15]:
overall_simple_chain = SimpleSequentialChain(
    chains=[chain_one, chain_two], 
    verbose=True
)

In [16]:
product = "Queen Size Sheet"
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
 "Royal Comfort: The Premier Provider of Queen Size Sheets"

This name effectively conveys that the company specializes in producing high-quality queen size sheets, and it also evokes a sense of luxury and elegance, making it an appealing choice for potential customers in search of premium bedding options.
 "Royal Comfort: Luxurious Queen-Size Sheet Specialists, Setting the Standard for Premium Bedding."

> Finished chain.


' "Royal Comfort: Luxurious Queen-Size Sheet Specialists, Setting the Standard for Premium Bedding."'

Simple Sequential Chain works well when there is a single input and single output.

### SequentialChain

![Sequential Chain](data/images/sequential-chain-figure.png)

In [17]:
from langchain.chains import SequentialChain

In [18]:
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english: "
    "{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, output_key="English_Review")

In [19]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )

In [20]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:"
    "{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, 
                       prompt=third_prompt,
                       output_key="language"
                      )

In [21]:
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "Summary: {summary}"
    "Language: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )

In [22]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [24]:
review = df.review[5]
overall_chain.invoke(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur... Vieux lot ou contrefaçon !?",
 'English_Review': ' I find the taste mediocre. The foam does not hold, which is strange. I buy the same ones in the store and the taste is much better... Old stock or counterfeit!?\n\nNote: This translation is based on the provided review in French. However, as a assistant, I can only provide a general translation and cannot confirm the validity of the suspicion of old stock or counterfeit.',
 'summary': ' The reviewer finds the taste of the product mediocre, and is concerned that it may be old stock or counterfeit due to the foam not holding and finding it inferior to the same product bought in-store.',
 'followup_message': " Cher rédacteur de la revue,\n\nTout d'abord, permettez-moi de vous remercier pour votre feedback honnête sur notre produit. Je suis désolé d'apprendre que vous avez trouvé le goût médiocr

## Router Chain

![Router Chain](data/images/router-chain.png)

In [25]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [26]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [27]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [28]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm_nvidia, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [29]:
print(destinations_str)

physics: Good for answering questions about physics
math: Good for answering math questions
History: Good for answering history questions
computer science: Good for answering computer science questions


In [30]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [31]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a 
language model select the model prompt best suited for the input.
You will be given the names of the available prompts and a 
description of what the prompt is best suited for. 
You may also revise the original input if you think that revising
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": name of the prompt to use or "DEFAULT"
    "next_inputs": string a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [32]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [33]:
print(router_template.format(input="What is 1+1?"))

Given a raw text input to a 
language model select the model prompt best suited for the input.
You will be given the names of the available prompts and a 
description of what the prompt is best suited for. 
You may also revise the original input if you think that revising
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{
    "destination": name of the prompt to use or "DEFAULT"
    "next_inputs": string a potentially modified version of the original input
}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is notwell suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
physics: Good for answering questions about physics
math: Good for answering math questions
History: Good for an

In [34]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [35]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


" Black body radiation is the electromagnetic radiation emitted by an idealized object, called a black body, that fully absorbs and emits all frequencies of radiation. It is a fundamental concept in physics and thermal radiation, describing the emission spectrum of a perfect radiator. The radiation emitted by a black body is a continuous spectrum that depends only on the temperature of the object, and follows Planck's law of black body radiation. This radiation includes visible light, as well as infrared and ultraviolet radiation.\n\nIt is important to note that real objects do not perfectly absorb and emit all frequencies of radiation, and therefore their radiation characteristics will differ somewhat from the idealized black body. Nevertheless, black body radiation serves as a valuable reference for understanding the radiation properties of various objects and materials."

In [36]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


' To answer the question "what is 2 + 2," I can break it down into its component parts, perform the addition operation on each part, and then put the results together to find the final answer.\n\nThe first component part is the number 2. The second component part is also the number 2. The addition operation tells us to combine these two numbers to find the sum. When we add 2 and 2, we get the number 4.\n\nTherefore, the answer to the question "what is 2 + 2" is 4. This is a simple example, but the process of breaking down a problem into its component parts and then putting the results together can be helpful for solving more complex math problems as well.'

In [37]:
chain.run("Who was Alexandar Hamilton?")



> Entering new MultiPromptChain chain...
History: {'input': 'Who was Alexander Hamilton?'}
> Finished chain.


" Alexander Hamilton (1755-1804) was a founding father of the United States, who served as the first Secretary of the Treasury from 1789 to 1795. He was born on the island of Nevis in the Caribbean and was orphaned at a young age. Through his own intelligence and hard work, he was able to attend college in the United States and eventually became a key figure in the movement for American independence.\n\nHamilton was a strong proponent of a strong central government and played a key role in the drafting of the U.S. Constitution. He was also a leading advocate for the creation of a national bank and the development of a system of public credit. These policies helped to establish the financial stability of the new nation and set the stage for its future economic growth.\n\nIn addition to his role as a statesman, Hamilton was also a prominent lawyer and writer. He was a co-author of the Federalist Papers, a series of essays that helped to secure the ratification of the U.S. Constitution. H